In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import fastparquet
import geopandas as gpd
import plotly.express as px
from plotly.offline import iplot
%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
path="F:/1_Bachelorarbeit/1_Daten/1_Yellow_Cab_Trips_2009-June_2016/6_Dataset_after_Exploration_and_Cleaning/After_Exploration_and_Cleaning_without_false_Entry.parquet"
# Testing the Parquet Files
df=pd.read_parquet(path=path,engine="fastparquet")
df.head()

,Pickup_DateTime,Passenger_Count,Pickup_Lon,Pickup_Lat,Fare_Amt,geometry_pickup,zone_pickup,LocationID_pickup,borough_pickup,Dropoff_Lon,Dropoff_Lat,geometry_dropoff,zone_dropoff,LocationID_dropoff,borough_dropoff,is_haversine,is_calculated_fare_amt_haversine,is_calculated_fare_amt_normal
index,,,,,,,,,,,,,,,,,,
183041,2009-01-01 00:13:43,2,-74.009254,40.706161,15.000000,POINT (-74.00925445556641 40.70616149902344),Financial District North,87,Manhattan,-73.995110,40.760044,POINT (-73.99510955810547 40.76004409790039),Clinton East,48,Manhattan,False,False,False
181137,2009-01-01 00:15:51,1,-73.940910,40.792622,15.253906,POINT (-73.94091033935547 40.79262161254883),East Harlem North,74,Manhattan,-73.923622,40.743958,POINT (-73.92362213134766 40.74395751953125),Sunnyside,226,Queens,False,False,True
241421,2009-01-01 00:16:50,4,-73.981300,40.768108,11.398438,POINT (-73.98130035400391 40.76810836791992),Lincoln Square East,142,Manhattan,-73.992531,40.751957,POINT (-73.99253082275391 40.75195693969727),Garment District,100,Manhattan,False,False,False
222602,2009-01-01 00:16:56,1,-73.998840,40.719589,5.398438,POINT (-73.99884033203125 40.71958923339844),Little Italy/NoLiTa,144,Manhattan,-74.006203,40.714466,POINT (-74.00620269775391 40.7144660949707),TriBeCa/Civic Center,231,Manhattan,False,False,False
30701,2009-01-01 00:17:08,2,-73.965240,40.770863,3.750000,POINT (-73.96524047851563 40.7708625793457),Upper East Side South,237,Manhattan,-73.971161,40.766846,POINT (-73.97116088867188 40.766845703125),Central Park,43,Manhattan,False,False,True


In [3]:
df.describe()

,Passenger_Count,Pickup_Lon,Pickup_Lat,Fare_Amt,LocationID_pickup,Dropoff_Lon,Dropoff_Lat,LocationID_dropoff
count,1.895135e+06,1.895135e+06,1.895135e+06,1.895135e+06,1.895135e+06,1.895135e+06,1.895135e+06,1.895135e+06
mean,1.688334e+00,-7.397551e+01,4.075108e+01,1.150741e+01,1.607103e+02,-7.397466e+01,4.075147e+01,1.589386e+02
std,1.310493e+00,3.391661e-02,2.653235e-02,8.826110e+00,6.646180e+01,3.256248e-02,3.051131e-02,6.930471e+01
min,1.000000e+00,-7.423862e+01,4.052285e+01,3.000039e+00,2.000000e+00,-7.423925e+01,4.050714e+01,1.000000e+00
25%,1.000000e+00,-7.399219e+01,4.073685e+01,6.101562e+00,1.130000e+02,-7.399150e+01,4.073580e+01,1.070000e+02
50%,1.000000e+00,-7.398209e+01,4.075359e+01,8.616387e+00,1.610000e+02,-7.398048e+01,4.075403e+01,1.610000e+02
75%,2.000000e+00,-7.396838e+01,4.076775e+01,1.300000e+01,2.310000e+02,-7.396512e+01,4.076862e+01,2.310000e+02
max,6.000000e+00,-7.370598e+01,4.091044e+01,1.100000e+02,2.630000e+02,-7.370129e+01,4.091263e+01,2.630000e+02


### Drop Columns for initial State:
#### We know the Time, where we are and where we want to go 

In [4]:
# Drop Columns
df.drop(columns=["is_haversine","is_calculated_fare_amt_haversine","is_calculated_fare_amt_normal"],inplace=True)

In [5]:
df.head()

,Pickup_DateTime,Passenger_Count,Pickup_Lon,Pickup_Lat,Fare_Amt,geometry_pickup,zone_pickup,LocationID_pickup,borough_pickup,Dropoff_Lon,Dropoff_Lat,geometry_dropoff,zone_dropoff,LocationID_dropoff,borough_dropoff
index,,,,,,,,,,,,,,,
183041,2009-01-01 00:13:43,2,-74.009254,40.706161,15.000000,POINT (-74.00925445556641 40.70616149902344),Financial District North,87,Manhattan,-73.995110,40.760044,POINT (-73.99510955810547 40.76004409790039),Clinton East,48,Manhattan
181137,2009-01-01 00:15:51,1,-73.940910,40.792622,15.253906,POINT (-73.94091033935547 40.79262161254883),East Harlem North,74,Manhattan,-73.923622,40.743958,POINT (-73.92362213134766 40.74395751953125),Sunnyside,226,Queens
241421,2009-01-01 00:16:50,4,-73.981300,40.768108,11.398438,POINT (-73.98130035400391 40.76810836791992),Lincoln Square East,142,Manhattan,-73.992531,40.751957,POINT (-73.99253082275391 40.75195693969727),Garment District,100,Manhattan
222602,2009-01-01 00:16:56,1,-73.998840,40.719589,5.398438,POINT (-73.99884033203125 40.71958923339844),Little Italy/NoLiTa,144,Manhattan,-74.006203,40.714466,POINT (-74.00620269775391 40.7144660949707),TriBeCa/Civic Center,231,Manhattan
30701,2009-01-01 00:17:08,2,-73.965240,40.770863,3.750000,POINT (-73.96524047851563 40.7708625793457),Upper East Side South,237,Manhattan,-73.971161,40.766846,POINT (-73.97116088867188 40.766845703125),Central Park,43,Manhattan


In [6]:
# Convert to parquet for quick reloading
# Set Path for Parquet Files
path="F:/1_Bachelorarbeit/1_Daten/1_Yellow_Cab_Trips_2009-June_2016/7_Preprocessing"
df.to_parquet(os.path.join(path, "With_Feature_Engineering_Without_false_Entry" + "." + "parquet"),engine="fastparquet")